# Importing the dependencies

In [109]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Loading the Images
Here, we will be defining the function which will load all the images from the generated dataset and return the images and their corresponding labels.

In [110]:
def load_images_from_directory(base_dir):
    data = []
    labels = []
    shapes = ['line', 'circle', 'ellipse', 'rectangle', 'rounded_rectangle', 'regular_polygon', 'star']
    
    for shape in shapes:
        shape_dir = os.path.join(base_dir, shape)
        for filename in os.listdir(shape_dir):
            img_path = os.path.join(shape_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if(img is not None):
                img = cv2.resize(img, (128, 128))
                data.append(img)
                labels.append(shape)
    
    return np.array(data), np.array(labels)

Let us now go ahead and load the data

In [111]:
data_dir = '../dataset'  # Path to the generated dataset
data, labels = load_images_from_directory(data_dir)

# Preprocessing the data
Here, we will be defining the function which will preprocess the data. We will be normalizing the images and one-hot encoding the labels.

In [112]:
def preprocess(data,labels):
    data = data.reshape((data.shape[0], 128, 128, 1)) / 255.0 # Normalize the data
    label_encoder = LabelEncoder() # Encode the labels
    labels = to_categorical(label_encoder.fit_transform(labels)) # One-hot encode the labels
    return data,labels,label_encoder

In [113]:
x, y,label_encoder = preprocess(data, labels)

# Splitting the data into training and test set

In [114]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)

# Defining the model architecture using CNN

## Layer-wise Breakdown:

**Input Layer (`input_shape=(128, 128, 1)`)**

* This defines the shape of the images the model will receive: 128 pixels wide, 128 pixels high, and 1 channel (grayscale).

**Convolutional Layers (`Conv2D`)**

* We have 5 convolutional layers with increasing numbers of filters (32, 64, 128, 256, 512).
* Each layer uses a 3x3 filter size (`(3, 3)`) to detect local patterns in the image.
* The `ReLU` activation function introduces non-linearity, allowing the model to learn complex relationships between features.

**Max Pooling Layers (`MaxPooling2D`)**

* Each convolutional layer is followed by a max-pooling layer with a pool size of 2x2 (`(2, 2)`).
* Max pooling reduces the spatial dimensions of the feature maps, helping to make the model more robust to small shifts and variations in the shapes' positions.

**Flatten Layer (`Flatten`)**

* This layer converts the multidimensional output of the final convolutional layer into a single-dimensional vector, preparing it for the fully connected layers.

**Dense Layers (`Dense`)**

* The first dense layer has 512 neurons and uses the `ReLU` activation function. It performs high-level feature extraction and pattern recognition.
* The final dense layer has `len(label_encoder.classes_)` neurons (the number of shape classes) and uses the `softmax` activation function. Softmax outputs probabilities for each class, allowing us to interpret the model's prediction as the class with the highest probability.

**Dropout Layer (`Dropout(0.8)`)**

* This layer randomly drops out (sets to zero) 80% of the neurons during training.
* Dropout acts as a regularization technique, preventing overfitting by forcing the network to learn more robust features and reducing reliance on any single neuron.

## Justification for 0.8 Dropout

* **Addressing Overfitting:** The relatively high dropout rate of 0.8 is likely chosen to combat overfitting, which can be a concern when training deep convolutional neural networks.
* **Empirical Choice:** The specific value of 0.8 might have been determined through experimentation and validation on your dataset. It's possible that lower dropout rates were tried but resulted in worse performance on the validation or test set.

## Overall Model Design Philosophy

The model follows a common pattern in image classification tasks:

1. **Feature Extraction:** The convolutional and pooling layers extract increasingly complex features from the images, starting with simple edges and corners and progressing to more abstract representations of shapes.
2. **Classification:** The dense layers, along with the softmax activation, perform the final classification based on the extracted features.
3. **Regularization:** Dropout helps prevent overfitting and improve the model's generalization ability.


In [115]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.8),
    Dense(len(label_encoder.classes_), activation='softmax')
])

/Users/dhruv/Desktop/Adobe_GenSolve/CURVETOPIA_DA/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compiling the model

In [116]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model

In [117]:
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [118]:
model.fit(x_train, y_train, epochs=25, validation_split=0.2, batch_size=32,callbacks=[early_stop])

Epoch 1/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 80s 532ms/step - accuracy: 0.2333 - loss: 1.8266 - val_accuracy: 0.5227 - val_loss: 1.0968
Epoch 2/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 82s 552ms/step - accuracy: 0.4911 - loss: 1.1807 - val_accuracy: 0.6504 - val_loss: 0.6573
Epoch 3/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 80s 539ms/step - accuracy: 0.6371 - loss: 0.7611 - val_accuracy: 0.6908 - val_loss: 0.6122
Epoch 4/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 76s 512ms/step - accuracy: 0.7166 - loss: 0.5925 - val_accuracy: 0.7706 - val_loss: 0.4255
Epoch 5/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 76s 508ms/step - accuracy: 0.7452 - loss: 0.4795 - val_accuracy: 0.7714 - val_loss: 0.4167
Epoch 6/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 75s 504ms/step - accuracy: 0.7741 - loss: 0.4302 - val_accuracy: 0.7950 - val_loss: 0.3778
Epoch 7/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 80s 536ms/step - accuracy: 0.7949 - loss: 0.3930 - val_accuracy: 0.8092 - val_loss: 0.3410
Epoch 8/25
149/149 ━━━━━━━━━━━━━━━━━━━━ 80s 540ms/step - accuracy: 0.7937 - loss: 0

# Evaluating the model

In [119]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")
print(f"Loss on test data: {loss * 100:.2f}%")

33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - accuracy: 0.8359 - loss: 0.2958
Accuracy on test data: 84.67%
Loss on test data: 29.13%


## Performance Evaluation and Rationale:

* Test Accuracy of 84.67%: This accuracy indicates the model's strong capability in correctly classifying unseen shape images, showcasing its effective generalization to new data.

* Loss of 29.13%: While there is room for further improvement, this loss value suggests the model's predictions are reasonably aligned with the true labels.

# Saving the model

In [120]:
model.save('../models/shapes_created_model.h5')